# Categorizing Chicago Communities using Crime Data and Foursquare

## Table of contents
* [Introduction / Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

## Introduction / Business Problem <a name="introduction"></a>

The objective of this project is to obtain the safest communities taking into account the crime rate. The result of this study is aimed at those interested in living in the city of Chicago.

Chicago is one of the largest cities of the united states with a population of over 2 and a half million people. Chicago has 77 communities grouped into 9 districts.  The city has reported more than 7 million crimes of every category since 2001, 259 thousand just in 2019.  Business Problem  We will deal with the decision of “Which community has had the least crimes in 2019”, finding the right community to move into or beginning a business entrepreneurship based on the security and venue(residence) density in each community. 
 

## Data <a name="data"></a>

Our study will be based on the data extracted from:
 * Crime Data(using the datasets provided by <a target='new' href='https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-present-Dashboard/5cd6-ry5g'>the city of chicago</a>)
 * Community information(scraped from <a target='new' href='https://en.wikipedia.org/wiki/Community_areas_in_Chicago'>this wikipedia page</a>)
 	* Using geopy to get each coordinate
 * Foursquare api to explore residences near our communities
 
**We will be using only 2019 crime data**.
 

In [1]:
import pandas as pd
import numpy as np
import requests
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
!pip install lxml
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Solving environment: done

# All requested packages already installed.



**Since we downloaded the data corresponding to 2019 into csv file named: ChicagoCrimes2019.csv we proceed to utilize it**

In [2]:
df_crimes = pd.read_csv('ChicagoCrimes2019.csv')
df_crimes.shape

(259004, 22)

In [107]:
df_crimes.head()

,CommunityArea,Latitude,Longitude
0,22.0,41.920626,-87.699237
1,23.0,41.900916,-87.718184
2,23.0,41.899790,-87.720116
3,30.0,41.840016,-87.712231
4,23.0,41.907072,-87.731331


**Crime Data preliminar cleaning**

In [3]:
#remove unnecessary columns
df_crimes.drop(df_crimes.columns.difference(['Community Area','Latitude','Longitude']), 1, inplace=True)
#rename for ease of access
df_crimes = df_crimes.rename(columns={"Community Area": "CommunityArea"})
#delete rows that dont have a community
df_crimes = df_crimes[df_crimes.CommunityArea != 0]
df_crimes.shape

(259002, 3)

Now we need a list of all communities, we use pandas to scrape the data from wikipedia

In [60]:
communities_raw_data = pd.read_html('https://en.wikipedia.org/wiki/Community_areas_in_Chicago')
# the pandas read_html function reads all tables from the scraped webpage. Since chicago has 77 communities divided into 9 districts 
#we take the first 9 indexes
print('the object communities_raw_data is of type {}, has {} items of type {}'.format(type(communities_raw_data),
                                                                                      len(communities_raw_data),
                                                                                      type(communities_raw_data[0])))

the object communities_raw_data is of type <class 'list'>, has 12 items of type <class 'pandas.core.frame.DataFrame'>


**Now that we have a list of DataFrames containing all the community data, we proceed to merge each item into only 1 DataFrame**

In [53]:
# define the dataframe columns
column_names = ['CommunityNumber', 'CommunityAreaName'] 

# instantiate the dataframe
community_data = pd.DataFrame(columns=column_names)

for index in range(9):
    for r,data in communities_raw_data[index].iterrows():
        number =data.Number
        name = data['Community area']

        community_data = community_data.append({'CommunityNumber': number,
                                              'CommunityAreaName': name} , ignore_index=True)

community_data.head()

,CommunityNumber,CommunityAreaName
0,8,Near North Side
1,32,Loop
2,33,Near South Side
3,5,North Center
4,6,Lake View


### We need a function to obtain the location of each community 

In [87]:
#returns the latitud and longitude of a given community
def get_community_location(community):
    address = community+', Chicago, IL'
    geolocator = Nominatim(user_agent="chicago_explorer")
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    return latitude,longitude

The next step is loop thru all communities and get each latitud,longitude


In [105]:
for i,data in community_data.iterrows():
    location= get_community_location(data.CommunityAreaName)
    community_data.at[i,'Latitude'] = location[0]
    community_data.at[i,'Longitude'] = location[1]
    
community_data.head()

,CommunityNumber,CommunityAreaName,Latitude,Longitude
0,8,Near North Side,41.900033,-87.634497
1,32,Loop,41.875562,-87.624421
2,33,Near South Side,41.856700,-87.624774
3,5,North Center,41.956107,-87.679160
4,6,Lake View,41.947050,-87.655429


## Methodology <a name="methodology"></a>

In [103]:
get_community_location('Near North Side')


(41.9000327, -87.6344975)

In [59]:
#df_crimes.loc[df_crimes['CommunityArea']==0]
df_crimes

,CommunityArea,Latitude,Longitude
0,22.0,41.920626,-87.699237
1,23.0,41.900916,-87.718184
2,23.0,41.899790,-87.720116
3,30.0,41.840016,-87.712231
4,23.0,41.907072,-87.731331
...,...,...,...
258999,27.0,41.877268,-87.711537
259000,25.0,41.874624,-87.745053
259001,24.0,NaN,NaN
259002,68.0,41.779174,-87.653278


### Create a map of chicago and put every community in the map as markers

In [24]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=15)


    
map_toronto

import geocoder # import geocoder

# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]

In [10]:

dfCrimes = pd.read_csv('ChicagoCrimes2019.csv')

In [11]:
dfCrimes.shape

(259004, 22)

In [17]:
dfCrimes['Community Area'].unique()


array([22., 23., 30., 31., 26., 18., 24., 25.,  5., 20., 14., 29., 53.,
       69., 15., 44., 75., 43., 27., 42., 54.,  4., 39., 60.,  1., 63.,
       49., 76.,  8., 73., 58., 67., 19.,  3., 46., 38.,  7., 28., 65.,
        6., 71., 66., 56., 55., 45., 68., 35., 33., 32., 48., 72., 36.,
       50., 41., 13., 64., 47., 16., 70., 21., 62., 51., 61., 77., 57.,
        2., 40., 34., 11., 52., 74., 59., 17., 10.,  9., 37., 12.,  0.,
       nan])

In [14]:
dfCrimes.head()

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,11965029,JD132142,01/01/2019 12:00:00 AM,028XX W SHAKESPEARE AVE,1752,OFFENSE INVOLVING CHILDREN,AGGRAVATED CRIMINAL SEXUAL ABUSE BY FAMILY MEMBER,APARTMENT,False,True,...,1.0,22.0,17,1156820.0,1914304.0,2019,04/20/2020 03:47:53 PM,41.920626,-87.699237,"(41.920626036, -87.699236545)"
1,11752916,JC342540,01/01/2019 12:00:00 AM,036XX W THOMAS ST,1752,OFFENSE INVOLVING CHILDREN,AGGRAVATED CRIMINAL SEXUAL ABUSE BY FAMILY MEMBER,APARTMENT,False,False,...,27.0,23.0,17,1151715.0,1907084.0,2019,04/19/2020 03:44:21 PM,41.900916,-87.718184,"(41.900915738, -87.718183718)"
2,11739188,JC326320,01/01/2019 12:00:00 AM,010XX N RIDGEWAY AVE,1752,OFFENSE INVOLVING CHILDREN,AGGRAVATED CRIMINAL SEXUAL ABUSE BY FAMILY MEMBER,APARTMENT,False,True,...,27.0,23.0,17,1151192.0,1906670.0,2019,04/19/2020 03:44:21 PM,41.899790,-87.720116,"(41.899789956, -87.720115618)"
3,11737723,JC323962,01/01/2019 12:00:00 AM,028XX S ST LOUIS AVE,1562,SEX OFFENSE,AGGRAVATED CRIMINAL SEXUAL ABUSE,APARTMENT,False,True,...,22.0,30.0,17,1153496.0,1884904.0,2019,04/19/2020 03:44:21 PM,41.840016,-87.712231,"(41.840016176, -87.712231415)"
4,11992909,JD166116,01/01/2019 12:00:00 AM,014XX N KEELER AVE,1562,SEX OFFENSE,AGGRAVATED CRIMINAL SEXUAL ABUSE,RESIDENCE,False,False,...,26.0,23.0,17,1148120.0,1909302.0,2019,03/23/2020 03:47:32 PM,41.907072,-87.731331,"(41.907072136, -87.731331357)"


In [20]:
communities_data = pd.read_html('https://en.wikipedia.org/wiki/Community_areas_in_Chicago')

In [23]:
len(communities_data)


12

In [32]:
communities_data[0]

,Number,Community area,Neighborhoods
0,8,Near North Side,Cabrini–Green The Gold Coast Goose Island Magn...
1,32,Loop,Loop New Eastside South Loop West Loop Gate
2,33,Near South Side,Dearborn Park Printer's Row South Loop Prairie...
